# CAMMAC https://cammac.readthedocs.io
S.Sénési for Météo-France - sept 2019 to march 2021


# Check ranges for various variables and experiments

## Print user-chosen field statistics ot user-chose time statistics for a series of variables and experiments

## Parameters stand in first cell, and are either commented here or in the doc (see above)

##  Define lists of interesting variables and experiments

In [ ]:
experiments=["historical","ssp126","ssp245","ssp585","ssp119"]
#experiments=["piControl","historical","ssp126","ssp245","ssp585","ssp119"]
#experiments=["ssp126"]#,"ssp245","ssp585","ssp119"]

variables={
    "Amon": ["pr","tas","prw","evspsbl"], 
    "Lmon": ["mrro","mrso","mrsos"],
    "Omon": ["sos"],
    "yr"  : ["dday","drain"]
    #"day" : ["pr"]
    }

# Which field statistics and tme statistics should be computed (CDO syntax)
fld_stats=[ "fldpctl,5", "fldpctl,50", "fldpctl,95"]
tim_stats=[ "timmean"]

excluded_models     = []
included_models     = None   # If this is a list, only listed models will be used

data_versions_tag   = "20200918_plus_derived"
data_versions_dir   = "/data/ssenesi/CAMMAC/data/select_data_versions"

# Define location of derived variables (yearly stats of daily precip)
derived_variables_pattern  = "/data/ssenesi/CMIP6_derived_variables/${variable}"
derived_variables_pattern += "/${variable}_${table}_${model}_${experiment}_${realization}_${grid}_${version}_${PERIOD}.nc"

do_test=True

In [ ]:
if do_test :
    experiments=["historical","ssp126"]
    #experiments=["ssp245"]
    #experiments=["piControl"]
    #variables={"Amon": ["pr"]}
    #fld_stats=[ "fldpctl,50"]
    #tim_stats=[ "timmean"]
    included_models= ["CNRM-CM6-1"]


In [ ]:
from IPython.core.display import display, HTML, Image
display(HTML("<style>.container { width:100% !important; }</style>"))
import requests  # use pip or conda to install it if needed
import json
import glob
import re
import os
import sys
from CAMMAClib.ancillary   import feed_dic, amail
from CAMMAClib.mips_et_al import table_for_var_and_experiment, project_for_experiment, project_for_model,\
     institute_for_model, mip_for_experiment, models_for_experiments, models_for_experiments_multi_var, \
     read_versions_dictionnary
from climaf.api import *
from climaf.period import firstyears

In [ ]:
# Load some user settings, if available
settings_file=os.getenv("CAMMAC_USER_PYTHON_CODE_DIR",".")+'/cammac_user_settings.py'
if os.path.exists(settings_file) :
    exec(compile(open(settings_file).read(),settings_file,'exec'))

In [ ]:
def period_of_experiment(experiment,dperiod):
    if experiment[0:3]=="ssp" : return "2081-2100"
    if experiment=="historical": return "1995-2014"
    if experiment=="piControl" : return firstyears(dperiod,100)        
    raise ValueError("Cannot handle exeriment %s"%experiment)

In [ ]:
data_versions=read_versions_dictionnary(data_versions_tag, data_versions_dir)
stats=dict()
models=set()
deep=False
do_print=True
# Compute stats and oragnize dict as :
#    stats[table][variable][experiment][tsat][fstat][model]
for experiment in experiments :
    for table in variables :
        for variable in variables[table] :
            if do_print :
                print experiment,table,variable,
            models_variants=models_for_experiments_multi_var(data_versions,[(variable,table)],[experiment],excluded_models,included_models)
            #print table,variable, models_variants
            for model,variant in models_variants :
                if do_print : print model,
                project="CMIP6"
                #print experiment,table,variable,model,variant
                grid,version,dperiod=data_versions[experiment][variable][table][model][variant]
                base_dict=dict(project=project, experiment=experiment,
                        model=model, institute=institute_for_model(model),
                        period=period_of_experiment(experiment,dperiod), variable=variable, table=table, 
                        version=version, grid=grid,realization=variant)
                #
                # Compute stats
                for fstat in fld_stats :
                    for tstat in tim_stats :
                        if table != "Lmon" :
                            sfield=ccdo(ds(**base_dict),operator="%s -%s"%(tstat,fstat))
                        else:
                            sftlf_dict=base_dict.copy()
                            sftlf_dict.update(experiment="piControl", mip="CMIP",table="fx",period="fx",variable="sftlf")
                            sftlf=ds(**sftlf_dict)
                            cfile(sftlf,deep=deep)
                            #print sftlf.baseFiles()
                            #print ds(**base_dict).baseFiles()
                            sfield=ccdo2_flip(ds(**base_dict),sftlf,operator="%s -%s -ifthen -setvrange,100,110"%(tstat,fstat))
                        feed_dic(stats,cvalue(sfield,deep=deep),table,variable,experiment,tstat,fstat,model)
                        cdrop(sfield)
                models.add(model)
            if do_print : print

In [ ]:
models=list(models)
models.sort()
for table in sorted(stats.keys()):
    for variable in variables[table] :
        for tstat in tim_stats:
            print "\n",table,variable,tstat
            print 50*"-"
            print "%21s "%(21*" "),
            for experiment in experiments :
                print "|",
                print "%-10s"%experiment,
                for fld_stat in fld_stats[0:-1] :
                    print "%9s "%(9*" "),
            print
            print "%21s "%(21*" "),
            for experiment in experiments :
                print "|",
                for fld_stat in fld_stats :
                    l=len(fld_stat)
                    if l > 9 : f=fld_stat[l-9:]
                    else : f=fld_stat
                    print "%9s "%f,
            print
            for model in models :
                print "%-20s  "%model,
                for experiment in experiments :
                    print "|",
                    d=stats[table][variable][experiment][tstat]
                    for fld_stat in fld_stats :
                        if model in d[fld_stat]:
                            print "%9.2e "%d[fld_stat][model],
                        else:
                            print "%9s "%(9*" "),
                print
            print



In [ ]:
#stats